<a href="https://colab.research.google.com/github/RomaricKanyamibwa/Normalisation-de-concepts-medicaux/blob/master/word2vec.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip3 install --upgrade gensim

In [0]:

import pandas as pd
#import collections
from pprint import pprint  # pretty-printer
import gensim
from gensim.models import word2vec
from gensim.parsing.preprocessing import remove_stopwords
import logging

In [8]:
%pwd
%cd /home/
%ls -l

/home
total 0


In [9]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}


E: Package 'python-software-properties' has no installation candidate
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
··········
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
Please enter the verification code: Access token retrieved correctly.


In [10]:
!mkdir drive
!google-drive-ocamlfuse drive
!ls drive/
!cd drive/Coolab/
!ls -l

 code
'Colab Notebooks'
 Coolab
 deeplearningbook.pdf
 drive-download-20181103T154811Z-001.zip
 kusnerb15.pdf
 norm
 raw
 sujet-normalisation.odp
 tokenized
 Training_test
 Word2VecModels
 word2vec-sg-s0100-w08-m004-ns05-s0.001-a0.025-i05.zip
 word2vec-sg-s0200-w08-m004-ns05-s0.001-a0.025-i05.zip
total 4
drwxr-xr-x 2 root root 4096 Dec 11 21:36 drive


In [11]:
google_drive="drive/Coolab/"
df_disorders=pd.read_csv("drive/Coolab/Disorders.csv",names=["Code","Disorder","GenDisorder"],sep="\t") 
Codes=df_disorders['Code']#Y vector (outputs)
Disorders=df_disorders['Disorder']#X vector(test)
GenDisorders=df_disorders['GenDisorder']#X vector (train)

Disorders=[m+" "+n for m,n in zip(Disorders,GenDisorders)]
print(Disorders[30])
for i,j in enumerate(zip(Codes,Disorders)):
  if('postaural' in j):
    print(j)

postaural fistula postauricular fistula


In [0]:
def input_tokenize(input_list):
    """This method reads the input file which is in gzip format"""
    
    logging.info("reading file {0}...this may take a while".format(input_list))
    
    for i, line in enumerate (input_list): 
        if (i%10000==0):
            logging.info ("read {0} reviews".format (i))
        # do some pre-processing and return a list of words for each review text
        line_process=remove_stopwords(line)
        line_process=gensim.utils.simple_preprocess(line_process,min_len=3,max_len=30)
        if not line_process:
          print(line_process,"before preprocess:",line)
          #line_process=line.split()
        else:    
          yield line_process

In [22]:
logging.info ("Tokenization")
Disorders_corpus=(input_tokenize(Disorders))
GenDisorders_corpus=(input_tokenize(GenDisorders))
logging.info ("Done Tokenization")

Corpus=[[m]+n for m,n in zip(Codes,Disorders_corpus)]
#print(Disorders_corpus[30])

[] before preprocess: t(11;17)(q13;q21) t(11;17)(q13;q21)
[] before preprocess: t(1;3)(p36;q23-25) t(1;3)(p36;q23-25)
[] before preprocess: t(14;18)(q32;q21) t(14;18)(q32;q21)
[] before preprocess: t(12;22)(p13;q11) t(12;22)(p13;q11)
[] before preprocess: t(4;14) t(4;14)
[] before preprocess: t(16;16) t(16;16)
[] before preprocess: t(2;19)(p23;p13) t(2;19)(p23;p13)
[] before preprocess: t(15;17)(q22;q12) t(15;17)(q22;q12)
[] before preprocess: t(17;21)(q11;q22) t(17;21)(q11;q22)
[] before preprocess: t(1;2)(p22;p12) t(1;2)(p22;p12)
[] before preprocess: t(x;2)(q11-12;p23) t(x;2)(q11-12;p23)
[] before preprocess: t(1;14) t(1;14)
[] before preprocess: t(x;1)(p11.2;q21) t(x;1)(p11.2;q21)
[] before preprocess: t(11;12)(p15;p13) t(11;12)(p15;p13)
[] before preprocess: t(6;9)(p23;q34) t(6;9)(p23;q34.1)
[] before preprocess: t(6;9)(p23;q34.1) t(6;9)(p23;q34.1)
[] before preprocess: t(15;22)(p11;q11) t(15;22)(p11;q11)
[] before preprocess: t(4;22)(q12;q11) t(4;22)(q12;q11)
[] before preprocess

In [23]:
print("DisordrersLen:",len(Disorders))
print("CodesLen:",len(Codes))

DisordrersLen: 623509
CodesLen: 623509


In [0]:

#for i,k in enumerate(Corpus):
 # print(k)
  #if i>10:break

print("Iterator")
iter_Corpus=iter(Corpus)
for i,k in enumerate(iter_Corpus):
  if  i==30:
    print(k)

In [0]:
def gensim_demo(sentences):
    logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
    model = word2vec.Word2Vec(sentences, iter=20, min_count=1, size=300, workers=6)

    # get the word vector of "the"
    #print(model.wv['the'])

    # get the most common words
    print(model.wv.index2word[0], model.wv.index2word[1], model.wv.index2word[2])

    # get the least common words
    vocab_size = len(model.wv.vocab)
    print(model.wv.index2word[vocab_size - 1], model.wv.index2word[vocab_size - 2], model.wv.index2word[vocab_size - 3])

    # find the index of the 2nd most common word ("of")
#    print('Index of "of" is: {}'.format(model.wv.vocab['of'].index))

    # some similarity fun
    print(model.wv.similarity('fracture', 'bone'), model.wv.similarity('broken', 'arm'))

    # what doesn't fit?
    print(model.wv.doesnt_match("green blue red zebra".split()))

#    str_data = read_data(root_path + filename)
#    index_data = convert_data_to_index(str_data, model.wv)
#    print(str_data[:4], index_data[:4])

    # save and reload the model
    model.save("drive/Coolab/Disorders")
    return model



In [25]:

#print("size corpus:",len(list(iter_Corpus)))
logging.info ("Tokenization")
Disorders_corpus=(input_tokenize(Disorders))
GenDisorders_corpus=(input_tokenize(GenDisorders))
logging.info ("Done Tokenization")
#pense à cree des fichier avec les donées

class MyCorpus(object):
  def __init__(self,Codes,GenDisorders_corpus):
    self.Codes = Codes
    self.GenDisorders_corpus=GenDisorders_corpus

  def __iter__(self):
    for m,n in zip(self.Codes,self.GenDisorders_corpus):
      yield [m]+n

#Corpus=MyCorpus(Codes,GenDisorders_corpus)
Corpus=[[m]+n for m,n in zip(Codes,Disorders_corpus)]
print("Corpus")
for i,k in enumerate(Corpus):
  print(k)
  if i>5:break

iter_Corpus=(Corpus)

model=gensim_demo(Corpus)


[] before preprocess: t(11;17)(q13;q21) t(11;17)(q13;q21)
[] before preprocess: t(1;3)(p36;q23-25) t(1;3)(p36;q23-25)
[] before preprocess: t(14;18)(q32;q21) t(14;18)(q32;q21)
[] before preprocess: t(12;22)(p13;q11) t(12;22)(p13;q11)
[] before preprocess: t(4;14) t(4;14)
[] before preprocess: t(16;16) t(16;16)
[] before preprocess: t(2;19)(p23;p13) t(2;19)(p23;p13)
[] before preprocess: t(15;17)(q22;q12) t(15;17)(q22;q12)
[] before preprocess: t(17;21)(q11;q22) t(17;21)(q11;q22)
[] before preprocess: t(1;2)(p22;p12) t(1;2)(p22;p12)
[] before preprocess: t(x;2)(q11-12;p23) t(x;2)(q11-12;p23)
[] before preprocess: t(1;14) t(1;14)
[] before preprocess: t(x;1)(p11.2;q21) t(x;1)(p11.2;q21)
[] before preprocess: t(11;12)(p15;p13) t(11;12)(p15;p13)
[] before preprocess: t(6;9)(p23;q34) t(6;9)(p23;q34.1)
[] before preprocess: t(6;9)(p23;q34.1) t(6;9)(p23;q34.1)
[] before preprocess: t(15;22)(p11;q11) t(15;22)(p11;q11)
[] before preprocess: t(4;22)(q12;q11) t(4;22)(q12;q11)
[] before preprocess

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


zebra


In [26]:
model.most_similar('fracture')

#model.most_similar('C3714535')


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('bone', 0.5890805721282959),
 ('closed', 0.5669761300086975),
 ('dislocation', 0.5612232685089111),
 ('displaced', 0.5310590267181396),
 ('nondisplaced', 0.5301649570465088),
 ('open', 0.5202462077140808),
 ('healing', 0.48214638233184814),
 ('end', 0.4581301212310791),
 ('iiia', 0.4445427656173706),
 ('fractures', 0.4442555010318756)]

In [27]:
len(model.wv['postaural'])
#(model.wv.vocab)
model.wv.vector_size
len(model.wv.vectors)

314686